<a href="https://colab.research.google.com/github/EliabeBastosDias/analise-de-vinhos-portugueses/blob/main/nt_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Combined All the dataset in one place

# Labeling

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

BASE = "/kaggle/input"

paths = {
    "brain": "/kaggle/input/fetal-brain-anomalies-ultrasound-1/content/drive/MyDrive/Dataset/Fetal-Brain-1.8k",

    "fetal_planes": "/kaggle/input/fetal-planes-db-dataset/Fetal_Planes_DB",

    # Heart 4-chamber — include all three subfolders
    "heart_4ch_training": "/kaggle/input/fetal-heart-four-chamber-ultrasound-image/training/images",
    "heart_4ch_validation": "/kaggle/input/fetal-heart-four-chamber-ultrasound-image/validation/images",
    "heart_4ch_testing": "/kaggle/input/fetal-heart-four-chamber-ultrasound-image/testing/images",

    "heart_ft": "/kaggle/input/first-trimester-fetal-echocardiography/Fetal Echocardiography First Trimester",

    "nt_dataset": "/kaggle/input/dataset-for-fetus-framework/Dataset for Fetus Framework",

    "maternal_fetal_planes": "/kaggle/input/maternal-fetal-ultrasound-planes/dataset"
}


# ---------------------------------------------------------
# LABELING FUNCTION
# ---------------------------------------------------------
def get_label(img_path):
    p = img_path.lower()

    # -----------------------------
    # BRAIN (normal + abnormal)
    # -----------------------------
    if "fetal-brain-1.8k" in p or "fetal-brain-abnormalities" in p:
        if "hc" in p or "thal" in p:
            return "brain-hc"            # Combine HC + thalamic into a single class
        if "vent" in p:
            return "brain-ventricular"
        if "cerebell" in p:
            return "brain-cerebellum"
        return "unknown"

    # -----------------------------
    # HEART 4CH — ONLY images folder
    # -----------------------------
    if "fetal-heart-four-chamber-ultrasound-image" in p:
        parent_folder = os.path.basename(os.path.dirname(p))
        if parent_folder == "images":
            return "heart-4ch"

    # -----------------------------
    # FIRST TRIMESTER HEART (3V, 3VT, etc.)
    # -----------------------------
    if "fetal echocardiography first trimester" in p:
        if "/v sign" in p or "/x sign" in p:
            return "heart-3vt"
        if "/aorta" in p or "/flows" in p:
            return "heart-3v"
        if "/other" in p:
            return "heart-other"
        return "unknown"

    # -----------------------------
    # NT DATASET
    # -----------------------------
    if "dataset for fetus framework" in p:
        if "/standard" in p:
            return "mid-sagittal-standard"
        if "/non-standard" in p:
            return "mid-sagittal-nonstandard"
        return "unknown"

    # -----------------------------
    # FETAL PLANES DB
    # -----------------------------
    if "fetal_planes_db" in p or "fetal-planes-db-dataset" in p:
        if "fetal abdomen" in p:
            return "abdomen"
        if "fetal brain" in p:
            return "brain-hc"  # Combine HC + thalamic here
        if "fetal femur" in p:
            return "femur"
        if "fetal thorax" in p:
            return "thorax"
        if "maternal cervix" in p:
            return "cervix"
        return "unknown"

    # -----------------------------
    # maternal-fetal dataset
    # -----------------------------
    if "maternal-fetal-ultrasound-planes" in p:
        if "hc" in p or "trans-thalamic" in p:
            return "brain-hc"  # Combine HC + thalamic here
        if "trans-ventricular" in p:
            return "brain-ventricular"
        if "trans-cerebellum" in p:
            return "brain-cerebellum"
        return "unknown"

    return "unknown"


# ---------------------------------------------------------
# COLLECT IMAGES
# ---------------------------------------------------------
all_rows = []

def add_images(root):
    for r, d, f in os.walk(root):
        for file in f:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                full = os.path.join(r, file)
                label = get_label(full)
                all_rows.append([full, label])


for key, path in paths.items():
    if os.path.exists(path):
        print("Scanning:", path)
        add_images(path)
    else:
        print("Missing:", path)


# ---------------------------------------------------------
# SAVE CSV
# ---------------------------------------------------------
df = pd.DataFrame(all_rows, columns=["image_path", "label"])
csv_path = "/kaggle/working/final_labels.csv"
df.to_csv(csv_path, index=False)

print("DONE! Total images:", len(df))
df.head()


Missing: /kaggle/input/fetal-brain-anomalies-ultrasound-1/content/drive/MyDrive/Dataset/Fetal-Brain-1.8k
Scanning: /kaggle/input/fetal-planes-db-dataset/Fetal_Planes_DB
Scanning: /kaggle/input/fetal-heart-four-chamber-ultrasound-image/training/images
Scanning: /kaggle/input/fetal-heart-four-chamber-ultrasound-image/validation/images
Scanning: /kaggle/input/fetal-heart-four-chamber-ultrasound-image/testing/images
Scanning: /kaggle/input/first-trimester-fetal-echocardiography/Fetal Echocardiography First Trimester
Scanning: /kaggle/input/dataset-for-fetus-framework/Dataset for Fetus Framework
Scanning: /kaggle/input/maternal-fetal-ultrasound-planes/dataset
DONE! Total images: 25052


,image_path,label
0,/kaggle/input/fetal-planes-db-dataset/Fetal_Pl...,unknown
1,/kaggle/input/fetal-planes-db-dataset/Fetal_Pl...,unknown
2,/kaggle/input/fetal-planes-db-dataset/Fetal_Pl...,unknown
3,/kaggle/input/fetal-planes-db-dataset/Fetal_Pl...,unknown
4,/kaggle/input/fetal-planes-db-dataset/Fetal_Pl...,unknown
